In [2]:
# Basic Prompt Text Summarization
import os
from dotenv import load_dotenv
from langchain_google_genai import  ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import  ChatPromptTemplate
from langchain_core.output_parsers import  StrOutputParser

_ = load_dotenv()

speech = """
Today, I stand before you not just as the Prime Minister of Pakistan but as a 
representative of billions of people whose voices are often unheard. The world 
faces grave challenges—climate change, global inequality, and the erosion of human 
rights. In South Asia, peace remains elusive due to unresolved conflicts, particularly 
in Jammu and Kashmir, where people have been denied their right to self-determination. 
I urge this assembly to uphold justice and the principles of the United Nations Charter. 
Let us come together to combat injustice, protect our planet, and create a fairer world for future generations.
"""
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [3]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant to summarize text."),
    HumanMessage(content=f"Please provide a short concise summary of the following speech: \nTEXT: {speech}"),
])


response = llm.invoke(prompt.format_messages())
response.content

'The Prime Minister of Pakistan addressed the assembly, highlighting global challenges like climate change, inequality, and human rights violations. He focused on the lack of peace in South Asia due to unresolved conflicts, particularly in Jammu and Kashmir, where self-determination is denied. He urged the assembly to uphold justice and UN principles for a fairer future.\n'

In [4]:
# Prompt Template Summarization
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant to summarize text."),
    ("human", """
    Please provide a short concise summary of the following speech: \nTEXT: {speech}
    & translate the summary into {language} language
    """),
])

chain = prompt | llm

response = chain.invoke({"speech": speech, "language": "Turkish"})
response.content

"Okay, here's a concise summary of the speech, followed by its Turkish translation:\n\n**Summary:**\n\nThe Prime Minister of Pakistan addresses the UN, highlighting global challenges like climate change, inequality, and human rights violations. He emphasizes the need for peace in South Asia, particularly concerning the unresolved conflict in Jammu and Kashmir, where self-determination is denied. He calls for justice, environmental protection, and a fairer world, urging the assembly to uphold UN principles.\n\n**Turkish Translation:**\n\nPakistan Başbakanı BM'ye hitap ederek iklim değişikliği, eşitsizlik ve insan hakları ihlalleri gibi küresel zorluklara dikkat çekiyor. Güney Asya'da, özellikle kendi kaderini tayin hakkının reddedildiği Jammu ve Keşmir'deki çözülmemiş çatışma nedeniyle barışın gerekliliğini vurguluyor. Adalet, çevre koruma ve daha adil bir dünya için çağrıda bulunarak meclisi BM ilkelerini desteklemeye teşvik ediyor.\n"

In [5]:
# Stuff Documentation Chain:
from pypdf import PdfReader
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import ChatPromptTemplate

# Load the PDF content
pdfReader = PdfReader("Proposal.pdf")
text = ""

for i, page in enumerate(pdfReader.pages):
    content = page.extract_text()
    if content:
        text += content

# Create a Document object
document = [Document(page_content=text)]

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to summarize text."),
    ("human", """
    Please provide a short concise summary of the following document: \nTEXT: {docs}
    & translate into {language} language
    """),
])

# Load the summarization chain, ensuring document_variable_name matches the input variable in the prompt
chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    prompt=prompt,
    document_variable_name="docs",  # Ensure this matches the input variable in the prompt
    verbose=False
)

# Invoke the chain with the document
response = chain.invoke({"input_documents": document, "language" : "Turkish"})

# Print the response
print(response["output_text"])


Okay, here's a concise summary of the letter, followed by a Turkish translation:

**Summary:**

The author is writing to a person they deeply admire, acknowledging their academic achievements in applied physics. They express that their feelings have grown beyond admiration and that they are now in love with the person. They respect the recipient's goals and want to be open about their feelings, even if the feelings are not reciprocated. The author expresses their support regardless of the recipient's decision, whether it leads to a deeper relationship or a strong friendship. They hope to have an open conversation about the letter and respect the recipient's decision.

**Turkish Translation:**

**Özet:**

Yazar, uygulamalı fizikteki akademik başarılarını takdir ettiği bir kişiye yazıyor. Duygularının hayranlığın ötesine geçtiğini ve artık o kişiye aşık olduğunu ifade ediyor. Alıcının hedeflerine saygı duyuyor ve duyguları karşılıklı olmasa bile duyguları hakkında açık olmak istiyor. Yaz

In [6]:
# Map Reduce Summarization:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import ChatPromptTemplate

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.create_documents(texts=[text])

# Define the map prompt
map_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to summarize text."),
    ("human", """
    Please provide a short concise summary of the following text:
    TEXT: {text}
    Also, translate the summary into {language}.
    """),
])

# Define the reduce prompt
reduce_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant to summarize text."),
    ("human", """
    Combine the following summaries into a single concise summary:
    TEXT: {text}
    Also, translate the combined summary into {language}.
    """),
])

# Load the summarization chain
chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=reduce_prompt,
    verbose=False,
)

# Prepare inputs for the chain
inputs = {
    "input_documents": chunks,
    "language": "Turkish"
}

# Invoke the chain with the inputs
response = chain.invoke(inputs)

# Print the output summary
print(response["output_text"])

Okay, here's a combined concise summary of all the texts, followed by its Turkish translation:

**Combined Summary:**

This is a heartfelt letter from the sender to "My Cute Hijabie," expressing deep admiration for her academic achievements, particularly in applied physics, and revealing the growth of romantic feelings over time. The sender acknowledges the challenges of distance and busy lives but felt compelled to share these feelings openly, emphasizing their respect for the recipient's intelligence, character, and goals. They prioritize the recipient's happiness and comfort, assuring them that their support will remain unwavering regardless of the recipient's response. The sender is open to a romantic relationship but also values their existing bond and is prepared to accept a continued friendship, emphasizing the importance of open communication, understanding, and the enduring family connection. The letter concludes with wishes for the recipient's happiness and success.

**Turkis

In [7]:
# Refine Summarization:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.create_documents(texts=[text])

# Define the map prompt
map_prompt = PromptTemplate(
    input_variables=["text", "language"],
    template="""
    You are a helpful assistant. Your task is to summarize and translate the given text.
    1. Summarize the following text into a short and concise summary.
    2. Translate the summary into {language}.
    TEXT: {text}
    """
)

# Define the reduce (refine) prompt
reduce_prompt = PromptTemplate(
    input_variables=["text", "language"],
    template="""
    You are a helpful assistant. Your task is to combine summaries and translate them.
    1. Combine the following summaries into one concise and coherent summary.
    2. Translate the final summary into {language}.
    TEXT: {text}
    """
)

# Create the LLM chains for map and refine steps
map_chain = LLMChain(llm=llm, prompt=map_prompt)
refine_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Create the RefineDocumentsChain
chain = RefineDocumentsChain(
    initial_llm_chain=map_chain,
    refine_llm_chain=refine_chain,
    document_variable_name="text",
    initial_response_name="output_text",
    verbose=True,
)

# Prepare inputs for the chain
inputs = {
    "input_documents": chunks,
    "language": "Turkish"
}

# Run the chain
response = chain(inputs)

# Print the output summary
print(response["output_text"])

C:\Users\Dell\AppData\Local\Temp\ipykernel_19132\337215907.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  map_chain = LLMChain(llm=llm, prompt=map_prompt)
C:\Users\Dell\AppData\Local\Temp\ipykernel_19132\337215907.py:38: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/refine_docs_chain/
  chain = RefineDocumentsChain(
C:\Users\Dell\AppData\Local\Temp\ipykernel_19132\337215907.py:53: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain(inputs)




> Entering new RefineDocumentsChain chain...

> Finished chain.
Okay, here's the combined summary and its Turkish translation:

**Combined Summary:**

This letter emphasizes the importance of the family bond and expresses a desire to maintain it. The author invites open communication and values the recipient's feelings and opinions. They conclude by thanking the recipient for reading the letter and wishing them happiness and success, regardless of future circumstances.

**Turkish Translation:**

Bu mektup, aile bağının önemini vurguluyor ve bu bağı koruma arzusunu ifade ediyor. Yazar, açık iletişime davet ediyor ve alıcının duygu ve düşüncelerine değer veriyor. Mektubu okuduğu için alıcıya teşekkür ederek ve gelecekteki koşullar ne olursa olsun onlara mutluluk ve başarı dileyerek sonlandırıyor.

